In [15]:
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 10)
import pickle

import pandas as pd
from learners import *
from dataset import *
from explanations import *

In [2]:
df = pd.concat([pd.read_csv("givemecredit_train.csv", index_col=0), ])
y = 1 - df['SeriousDlqin2yrs'].reset_index(drop=True)
X = df.drop('SeriousDlqin2yrs', axis=1).fillna(0).reset_index(drop=True)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Get true model

In [4]:
true_mlp = train_MLP(X, y, rand_state=42000, batch_size=50, max_iter=600, rand_param=False)

random state is 42000
data size is (150000, 10)
(0.01, 0.001, 50, 600)


In [5]:
true_mlp.score(X, y)

0.9361133333333334

In [6]:
y = true_mlp.predict(X)

## Generate Explanation

In [7]:
neg_X, pos_X = separate(X, y)

In [8]:
NUM_CLUSTER = 20
neg_exp_X = fit_kmedoid(neg_X, int(len(neg_X) / NUM_CLUSTER))

In [9]:
def mix_up(ws, part=5):
    np.random.seed(42000)
    rand_idx = np.random.permutation(range(len(ws)))
    return [ws[idxs] for idxs in np.array_split(rand_idx, part)]

In [10]:
'''
Randomly split up positive to do clustering and then combine otherwise memory needed too large
5 is the smallest it can go
'''
pos_exp_X = np.vstack([fit_kmedoid(x, int(len(x) / NUM_CLUSTER)) for x in mix_up(pos_X, part=5)])

In [11]:
all_exp_X, all_exp_Y = combine(neg_exp_X, pos_exp_X)

## Compute the version space

In [12]:
PREFIX = "mlp_exp"

In [13]:
from multiprocessing import Pool

In [ ]:
MOTHER_SEED = [79821, 23234, 45645, 567567, 7868, 32432][3] #ensure randomness
all_vs = []
start_pct = 5
for i in range(9):
    pct = start_pct + 10 * i
    (allowed_X, allowed_y), (not_X, not_y) = mlp_explanation_policy(all_exp_X, all_exp_Y, true_mlp, cutoff_percentile=pct)
    
    per_proc = int(400 / 8)
    def train_consistent_mlps(num_proc): 
        return [train_MLP(allowed_X, allowed_y, rand_state=num_proc * per_proc + i, mother_seed=MOTHER_SEED, 
                          batch_size=50, max_iter=600, rand_param=True) for i in range(per_proc)]

    consistent_mlps = None
    with Pool(8) as pool:
        consistent_mlps = pool.map(train_consistent_mlps, range(8))
    consistent_mlps = sum(consistent_mlps, [])
    
    all_vs.append(consistent_mlps)
    pickle.dump(consistent_mlps, open(PREFIX + "mlp_models_pct_{}.p".format(pct), "wb"))

random state is 50random state is 0random state is 200random state is 100random state is 250
random state is 150random state is 350data size is (7123, 10)



